# **1. Initiall Instructions**

In [ ]:
!pip install -q rdkit
!pip install -q xgboost
!pip install -q optuna

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from xgboost import XGBClassifier
import optuna
import seaborn as sns
from joblib import dump

from rdkit import Chem
from rdkit.Chem.SaltRemover import SaltRemover
from rdkit.Chem.AllChem import GetMorganGenerator

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, matthews_corrcoef, make_scorer
from sklearn.model_selection import StratifiedKFold, cross_validate, train_test_split
from sklearn.neural_network import MLPClassifier

# **2) Data Preparation**

In [ ]:
data_ki = pd.read_csv(r'/content/drive/MyDrive/5ht7_Ki(1).csv', sep=';', usecols=[0,7,8,9,10,11,45])
df_ki = pd.DataFrame(data_ki)
df_ki = df_ki[df_ki['Action Type'].isin(['ANTAGONIST', 'INHIBITOR'])]
print(df_ki.shape)

data_ic50 = pd.read_csv(r'/content/drive/MyDrive/5ht7_IC50.csv', sep=';', usecols=[0,7,8,9,10,11,45])
df_ic50 = pd.DataFrame(data_ic50)
df_ic50 = df_ic50[df_ic50['Action Type'].isin(['ANTAGONIST', 'INHIBITOR'])]
df_ic50['Standard Value'] = df_ic50['Standard Value'] /2   #According to Kaliokoski et al. in "Comparability of Mixed IC50 Data – A Statistical Analysis", Ki value is approximately equal to IC50/2.
df_ic50['Standard Type'] = 'Ki'
print(df_ic50.shape)

df_conc = pd.concat([df_ki, df_ic50])
print(df_conc.shape)

In [ ]:
ser = df_conc.isna().sum()
out_ser = ser.sum()

print(f'> DataFrame shape: {df_conc.shape}')
print(f'> DataFrame columns: {df_conc.columns}')
if out_ser == 0:
  print('> DataFrame has no missing values')
else:
  print(f'> DataFrame has {out_ser} missing values: \n')
  print(ser)

In [ ]:
df_conc = df_conc.drop_duplicates(subset=['Molecule ChEMBL ID'], keep=False)
df_conc['Bin Activity'] = np.where(df_conc['Standard Value'] <= 50, 1, 0)
print(f'Shape after drop_duplicates: {df_conc.shape}')

In [ ]:
classes = df_conc['Bin Activity'].value_counts()

fig = plt.figure(figsize=(10,6))
plt.bar(classes.index, classes.values, color=['lightcoral', 'lightgreen'])
plt.xticks(classes.index)
plt.xlabel('Class')
plt.ylabel('Count of molecules')
plt.title('Classes distribution')
plt.text(classes.index[0], classes.values[0] / 2,
         f'{classes[0]} molecules', ha='center', fontsize=12, color='black')
plt.text(classes.index[1], classes.values[1] / 2,
         f'{classes[1]} molecules', ha='center', fontsize=12, color='black')
plt.grid(axis='y')
plt.gca().set_facecolor('#f0f0f0')
plt.tight_layout()

In [ ]:
remover = SaltRemover()
fps_list = [1024, 2048, 4096, 8192]
rds_list = list(range(1, 6))
list_of_indexes = []

scoring_metrics = {
    'accuracy': 'accuracy',
    'matthews_corrcoef': make_scorer(matthews_corrcoef),
}


dict_for_scores = {
    'acc': [],
    'mcc': [],
}


skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)


list_of_models = [RandomForestClassifier(random_state=42),
                  SVC(random_state=42),
                  LogisticRegression(random_state=42),
                  XGBClassifier(random_state=42),
                  MLPClassifier(random_state=42)]

# **3. Model and Fingerprints Pre-Screening**

In [ ]:
for fps in fps_list:
  for rds in rds_list:
    fps_gen = GetMorganGenerator(fpSize=fps, radius=rds)

    df_conc['fps'] = df_conc['Smiles'].apply(lambda x:
      fps_gen.GetFingerprint(mol) if (mol := Chem.MolFromSmiles(x)) and (mol := remover.StripMol(mol)) else None
      )


    X = np.array(df_conc['fps'].to_list())
    y = np.array(df_conc['Bin Activity'])

    for model in list_of_models:
      scores = cross_validate(model, X, y, cv=skf, scoring=scoring_metrics)

      dict_for_scores['acc'].append(scores['test_accuracy'].mean())
      dict_for_scores['mcc'].append(scores['test_matthews_corrcoef'].mean())
      list_of_indexes.append(f'{model.__class__.__name__}_fps={fps}, rds={rds}')

      print(f'{model.__class__.__name__} calculated (fps={fps}, rds={rds}). Running next model')

In [ ]:
df_out = pd.DataFrame(data=dict_for_scores, index=list_of_indexes)
row_with_max_acc = df_out.loc[df_out['acc'].idxmax()]
row_with_max_mcc = df_out.loc[df_out['mcc'].idxmax()]

print(row_with_max_acc)
print()
print(row_with_max_mcc)

# **4.Optuna Tuning**

In [ ]:
fps_gen = GetMorganGenerator(fpSize=4096, radius=1)  #adjust fpSize and radius based on the score from above fps/rds screening

df_conc['fps'] = df_conc['Smiles'].apply(lambda x:
      fps_gen.GetFingerprint(mol) if (mol := Chem.MolFromSmiles(x)) and (mol := remover.StripMol(mol)) else None
      )

X = np.array(df_conc['fps'].to_list())
y = np.array(df_conc['Bin Activity'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
xgb_model = XGBClassifier(random_state=42)
xgb_model.fit(X, y)

y_pred = xgb_model.predict(X_test)
test_acc = accuracy_score(y_test, y_pred)
test_mcc = matthews_corrcoef(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)

print(f'Test accuracy: {test_acc}')
print(f'Test MCC: {test_mcc:.2f}')

plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False, xticklabels=['Class 0', 'Class 1'], yticklabels=['Class 0', 'Class 1'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix as Heatmap')
plt.show()

In [ ]:
def objective(trial):

  n_estimators = trial.suggest_int('n_estimators', 50, 500)
  max_depth = trial.suggest_int('max_depth', 3, 10)
  learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3)
  subsample = trial.suggest_float('subsample', 0.5, 1.0)
  colsample_bytree = trial.suggest_float('colsample_bytree', 0.5, 1.0)
  gamma = trial.suggest_float('gamma', 0, 10)
  reg_alpha = trial.suggest_float('reg_alpha', 0, 10)
  reg_lambda = trial.suggest_float('reg_lambda', 0, 10)

  model = XGBClassifier(n_estimators=n_estimators, max_depth=max_depth, learning_rate=learning_rate, subsample=subsample,
                        colsample_bytree=colsample_bytree, gamma=gamma, reg_alpha=reg_alpha, reg_lambda=reg_lambda, random_state=42)

  scores = cross_validate(model, X, y, cv=skf, scoring=scoring_metrics)

  return scores['test_accuracy'].mean()

In [ ]:
sampler = optuna.samplers.TPESampler(seed=13)

study = optuna.create_study(direction='maximize', sampler=sampler)
study.optimize(objective, n_trials=200)

In [ ]:
best_params = study.best_params

n_estimators = best_params['n_estimators']
max_depth = best_params['max_depth']
learning_rate = best_params['learning_rate']
subsample = best_params['subsample']
colsample_bytree = best_params['colsample_bytree']
gamma = best_params['gamma']
reg_alpha = best_params['reg_alpha']
reg_lambda = best_params['reg_lambda']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
xgb_model_after_optuna= XGBClassifier(n_estimators=n_estimators,
                                      max_depth=max_depth,
                                      learning_rate=learning_rate,
                                      subsample=subsample,
                                      colsample_bytree=colsample_bytree,
                                      gamma=gamma,
                                      reg_alpha=reg_alpha,
                                      reg_lambda=reg_lambda,
                                      random_state=42)
xgb_model_after_optuna.fit(X_train, y_train)

y_pred = xgb_model_after_optuna.predict(X_test)
test_acc = accuracy_score(y_test, y_pred)
test_mcc = matthews_corrcoef(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)

print(f'Test accuracy: {test_acc}')
print(f'Test MCC: {test_mcc:.2f}')

plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=['Class 0', 'Class 1'], yticklabels=['Class 0', 'Class 1'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix as Heatmap')
plt.show()

# **5.Best Model Saving**

In [ ]:
#Select best model. In this case, base model was better than model after Optuna tuning
os.chdir('desired_location')

final_model = xgb_model
path = os.path.join(os.getcwd(), 'best_xgb_model.joblib')
dump(final_model, path)